## 1-Python Spark SQL and DataFrame  (Basic Example)
### This notebook is desinged to work on your local computer. 
### Note: Install pyspark package via Anaconda before running this code.

### Spark DataFrame Operations 

In [259]:
#Starting Point: SparkSession
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .getOrCreate()

In [260]:
spark

### 1-Read Data

In [261]:
# spark is an existing SparkSession
df = spark.read.csv('people.csv', inferSchema=True, header=True)

### 2-Inspect Data

In [262]:
# Displays the content of the DataFrame to stdout
df.show()

+-----+---+---------+
| name|age|      job|
+-----+---+---------+
|Jorge| 30|Developer|
|  Bob| 32|Developer|
+-----+---+---------+



In [263]:
# spark, df are from the previous example
# Print the schema in a tree format
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)



In [264]:
# Return df column names and data types
df.dtypes

[('name', 'string'), ('age', 'int'), ('job', 'string')]

In [265]:
# get the column names
df.columns

['name', 'age', 'job']

In [266]:
# returns the first row of the table
df.first()

Row(name='Jorge', age=30, job='Developer')

In [267]:
# returns top n rows  
df.head(2)

[Row(name='Jorge', age=30, job='Developer'),
 Row(name='Bob', age=32, job='Developer')]

In [268]:
df.toPandas().head(2)

name  age        job
0  Jorge   30  Developer
1    Bob   32  Developer

In [269]:
# returns top n rows  
df.take(2)

[Row(name='Jorge', age=30, job='Developer'),
 Row(name='Bob', age=32, job='Developer')]

In [270]:
# So what is difference between df.head(n) vs. df.take(n)?
# No difference. My take on this is Spark DataFrame try to mimic Python DataFrame hence, df.head(n) is introduced.  

In [271]:
# Display the content of df
df.show(2)

+-----+---+---------+
| name|age|      job|
+-----+---+---------+
|Jorge| 30|Developer|
|  Bob| 32|Developer|
+-----+---+---------+



In [272]:
# Descriptive Statistics of the data 
df.describe().show()

+-------+-----+------------------+---------+
|summary| name|               age|      job|
+-------+-----+------------------+---------+
|  count|    2|                 2|        2|
|   mean| null|              31.0|     null|
| stddev| null|1.4142135623730951|     null|
|    min|  Bob|                30|Developer|
|    max|Jorge|                32|Developer|
+-------+-----+------------------+---------+



In [273]:
df.toPandas().describe() # Pandas way

age
count   2.000000
mean   31.000000
std     1.414214
min    30.000000
25%    30.500000
50%    31.000000
75%    31.500000
max    32.000000

### 3-Query Data

### 3A. select()

In [274]:
# Show all entries in name column
df.select("name").show() 

+-----+
| name|
+-----+
|Jorge|
|  Bob|
+-----+



In [275]:
# or use df inside select
df.select(df["name"]).show()

+-----+
| name|
+-----+
|Jorge|
|  Bob|
+-----+



In [276]:
# Show all entries in name and age columns
df.select("name","age").show()

+-----+---+
| name|age|
+-----+---+
|Jorge| 30|
|  Bob| 32|
+-----+---+



In [277]:
df.select().show()

++
||
++
||
||
++



In [278]:
df.select("*").show()

+-----+---+---------+
| name|age|      job|
+-----+---+---------+
|Jorge| 30|Developer|
|  Bob| 32|Developer|
+-----+---+---------+



In [279]:
df.select(df["age"] > 31).show()
# try this and won't work: df.select("age" > 31).show() 

+----------+
|(age > 31)|
+----------+
|     false|
|      true|
+----------+



In [280]:
df.select(df.age > 31).show()

+----------+
|(age > 31)|
+----------+
|     false|
|      true|
+----------+



### 3B. where()

In [281]:
# Show all entries where age > 31
df.where("age > 31").show()

+----+---+---------+
|name|age|      job|
+----+---+---------+
| Bob| 32|Developer|
+----+---+---------+



### 3C. between()

In [282]:
# Show all entries where age > 31
df.select(df.age.between(30, 31)).show()

+-----------------------------+
|((age >= 30) AND (age <= 31))|
+-----------------------------+
|                         true|
|                        false|
+-----------------------------+



In [283]:
df.where(df.age.between(30, 31)).show()

+-----+---+---------+
| name|age|      job|
+-----+---+---------+
|Jorge| 30|Developer|
+-----+---+---------+



### 3D. when()

In [284]:
from pyspark.sql import functions as F

In [285]:
df.select("name", F.when(df.age > 30, 1)).show()

+-----+-------------------------------+
| name|CASE WHEN (age > 30) THEN 1 END|
+-----+-------------------------------+
|Jorge|                           null|
|  Bob|                              1|
+-----+-------------------------------+



In [286]:
df.select("name", F.when(df.age > 30, 1).otherwise(0)).show()

+-----+--------------------------------------+
| name|CASE WHEN (age > 30) THEN 1 ELSE 0 END|
+-----+--------------------------------------+
|Jorge|                                     0|
|  Bob|                                     1|
+-----+--------------------------------------+



### 3E. Running SQL Queries Programmatically
#### The sql function on a SparkSession enables applications to run SQL queries programmatically and returns the result as a DataFrame.

In [287]:
# Register the DataFrame as a SQL temporary view
df.createTempView("people")

In [288]:
df2 = spark.sql("SELECT * FROM people").show()

+-----+---+---------+
| name|age|      job|
+-----+---+---------+
|Jorge| 30|Developer|
|  Bob| 32|Developer|
+-----+---+---------+



#### Temporary views in Spark SQL are session-scoped and will disappear if the session that creates it terminates. If you want to have a temporary view that is shared among all sessions and keep alive until the Spark application terminates, you can create a global temporary view. 

In [289]:
# Register the DataFrame as a global temporary view
df.createGlobalTempView("people")

In [290]:
df2 = spark.sql("SELECT * FROM people").show()

+-----+---+---------+
| name|age|      job|
+-----+---+---------+
|Jorge| 30|Developer|
|  Bob| 32|Developer|
+-----+---+---------+



In [291]:
# Return the contents of df as Pandas DataFrame
dfp = df.toPandas()
dfp

name  age        job
0  Jorge   30  Developer
1    Bob   32  Developer

### 4. GroupBy 

In [292]:
# Group by age, count the members in the groups
df.groupBy("age") \
.count() \
.show()

+---+-----+
|age|count|
+---+-----+
| 32|    1|
| 30|    1|
+---+-----+



In [293]:
df.groupBy('age').min().show()

+---+--------+
|age|min(age)|
+---+--------+
| 32|      32|
| 30|      30|
+---+--------+



### 5. Aggregate

In [294]:
# Display minimum age
df.agg({'age': 'min'}).show()

+--------+
|min(age)|
+--------+
|      30|
+--------+



In [295]:
df.agg({'age': 'mean'}).show()

+--------+
|avg(age)|
+--------+
|    31.0|
+--------+



In [296]:
df.agg(F.mean('age').alias('Average Age')).show()

+-----------+
|Average Age|
+-----------+
|       31.0|
+-----------+



In [297]:
df.select([F.mean('age'), F.max('age'), F.min('age')]).show()

+--------+--------+--------+
|avg(age)|max(age)|min(age)|
+--------+--------+--------+
|    31.0|      32|      30|
+--------+--------+--------+



#### Display in 2 significant digits

In [298]:
df_summary = df.select([F.mean('age').alias('avg'), F.max('age').alias('max'), F.min('age').alias('min')])

In [299]:
from pyspark.sql.functions import format_number
df_summary.select([format_number('avg', 2).alias('avg'), \
                   format_number('max', 2).alias('max'), \
                   format_number('min', 2).alias('min')]).show()


+-----+-----+-----+
|  avg|  max|  min|
+-----+-----+-----+
|31.00|32.00|30.00|
+-----+-----+-----+



### 6. Sort

In [300]:
df.show()

+-----+---+---------+
| name|age|      job|
+-----+---+---------+
|Jorge| 30|Developer|
|  Bob| 32|Developer|
+-----+---+---------+



In [301]:
df.sort("age", ascending=False).show()

+-----+---+---------+
| name|age|      job|
+-----+---+---------+
|  Bob| 32|Developer|
|Jorge| 30|Developer|
+-----+---+---------+



In [302]:
df.sort("name", ascending=True).show()

+-----+---+---------+
| name|age|      job|
+-----+---+---------+
|  Bob| 32|Developer|
|Jorge| 30|Developer|
+-----+---+---------+



### 7. Filter

In [303]:
df.filter(df["age"]>31).show()

+----+---+---------+
|name|age|      job|
+----+---+---------+
| Bob| 32|Developer|
+----+---+---------+



In [304]:
df.orderBy('age').show()

+-----+---+---------+
| name|age|      job|
+-----+---+---------+
|Jorge| 30|Developer|
|  Bob| 32|Developer|
+-----+---+---------+



### 8. Handling NA

In [305]:
# show non-na rows  
df.na.drop().show()

+-----+---+---------+
| name|age|      job|
+-----+---+---------+
|Jorge| 30|Developer|
|  Bob| 32|Developer|
+-----+---+---------+



In [306]:
# show na rows
df.where(F.isnan("age")).show()

+----+---+---+
|name|age|job|
+----+---+---+
+----+---+---+



In [307]:
# other functions (may not apply here)

# 1-Fill all missing values with a constant value (e.g.,50) 
# df.na.fill(50).show()

# 2-Return new df replacing one value (e.g., 10) with another (e.g., 20)
# df.na.replace(10, 20).show() 


### 9. Spark DF --> Pandas DF 

In [308]:
dfp = df.toPandas()
dfp

name  age        job
0  Jorge   30  Developer
1    Bob   32  Developer

In [309]:
dfp.iloc[0,1] = 53

In [310]:
dfp

name  age        job
0  Jorge   53  Developer
1    Bob   32  Developer

### 10.  Pandas DF --> Spark DF

In [311]:
df_spark = spark.createDataFrame(dfp)
df_spark

DataFrame[name: string, age: bigint, job: string]

In [312]:
df_spark.dtypes

[('name', 'string'), ('age', 'bigint'), ('job', 'string')]

In [313]:
df.dtypes

[('name', 'string'), ('age', 'int'), ('job', 'string')]

In [314]:
# use this to have a specific schema
from pyspark.sql.types import *
mySchema = StructType([ StructField("name", StringType(), True), \
                        StructField("age", IntegerType(), True), \
                        StructField("job", StringType(), True)])

In [315]:
df_spark1 = spark.createDataFrame(dfp, schema=mySchema)

In [316]:
df_spark1.dtypes

[('name', 'string'), ('age', 'int'), ('job', 'string')]

### 11. Write & Save to Files

In [317]:
df.show()

+-----+---+---------+
| name|age|      job|
+-----+---+---------+
|Jorge| 30|Developer|
|  Bob| 32|Developer|
+-----+---+---------+



In [318]:
df.toPandas().to_csv('mycsv.csv')

In [248]:
# df \
#   .coalesce(1) \
#   .write \
#   .mode('overwrite') \
#   .option('header', 'true') \
#   .csv('output.csv')

In [321]:
# df.write.csv('mycsv1.csv',header=True)
#df.write.format('csv').save('mycsv2.csv',header=True)

In [258]:
# Stop SparkSession
spark.stop()